# k means ||

In [1]:
# import the python libraries to create/connect to a Spark Session
from pyspark.sql import SparkSession

# build a SparkSession 
#   connect to the master node on the port where the master node is listening (7077)
#   declare the app name 
#   configure the executor memory to 512 MB
#   either *connect* or *create* a new Spark Context
spark = SparkSession.builder \
    .master("spark://spark-master:7077")\
    .appName("prova iniziale")\
    .config("spark.executor.memory", "512m")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/30 10:50:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
# create a spark context
sc = spark.sparkContext

# print its status
sc

<SparkContext master=spark://spark-master:7077 appName=prova iniziale>

# Import data from sklearn



In [4]:
!pip install scikit-learn # to be run at the launch of "docker compose up" 

In [5]:
import numpy as np
import pandas as pd
import random
import sklearn.datasets #va installato

# from pyspark.sql.functions import sum
from pyspark.sql.functions import col
from pyspark.sql.functions import mean
from pyspark.sql.functions import stddev
from pyspark.ml.feature import StandardScaler

In [6]:
# Hyperparameters
k = 20
G = 3 # Giosu factor
len_df = 10_000

In [7]:
# Import dataframe directly from sklearn
prova =  sklearn.datasets.fetch_kddcup99(percent10 = True, as_frame = True)
X = prova.data

# prova = sc.parallelize(sklearn.datasets.fetch_kddcup99(percent10 = True, as_frame = True))

In [8]:
# Show the dataframe (to be deleted for a bigger dataframe) (TO BE DELETED)
X

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,b'tcp',b'http',b'SF',310,1881,0,0,0,0,...,86,255,1.0,0.0,0.01,0.05,0.0,0.01,0.0,0.0
494017,0,b'tcp',b'http',b'SF',282,2286,0,0,0,0,...,6,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0
494018,0,b'tcp',b'http',b'SF',203,1200,0,0,0,0,...,16,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0
494019,0,b'tcp',b'http',b'SF',291,1200,0,0,0,0,...,26,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0


# Create a Spark dataframe
And then acquire important informations about the dataframe: # columns, # rows, # partitions, the schema, ...

In [9]:
# Create the spark dataframe with a smaller chunk of data (just to work easily)
X_smaller = X.iloc[random.sample(range(1, len(X.index)), len_df)]
spark_X = spark.createDataFrame(X_smaller)

/usr/bin/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/bin/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [10]:
# Check the number of partitions the DataFrame is divided into the three workers yet
spark_X.rdd.getNumPartitions()

3

In [11]:
# Define a function that is able to select the single row in the dataframe
def getrows(df, rownums=None): # copied from the internet
    return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: x[0])

In [ ]:
# Define the number of rows and columns in the dataframe
n_rows = spark_X.count()
n_cols = len(getrows(spark_X, rownums=[0]).collect()[0])

In [ ]:
# Delete the non-numerical values in an easy way
# clean_X = spark_X.select('*').drop('protocol_type', 'service', 'flag')
# clean_X.printSchema()

# Delete the non-numerical values in a reusable way
col_type = np.array(spark_X.dtypes)
types = col_type[:,1] 
colnames = col_type[:,0]
clean_X = spark_X.select([col(colnames[i]) for i in range(len(colnames)) if not types[i] == 'binary'])
# clean_X.printSchema()

In [ ]:
# Normalize the numerical columns

i = 0
means = np.zeros(len(clean_X.columns))
devstd = np.zeros(len(clean_X.columns))
# Calculate mean and variance for each column
for column in clean_X.columns:
    means[i] = float(clean_X.select(mean(column)).head()[0])
    devstd[i] = float(clean_X.select(stddev(column)).head()[0])
    i = i+1

# norm_X = clean_X
means, devstd

TO BE NORMALIZED!!

# Step 0: Useful functions

In [ ]:
# # Define the function for the distance between the cluster centers and the pandas dataframe
# def distance(xrow,broadC):
#     '''Distance between a dataframe.row
#     and the broadcasted list of centers
#     in the form of dataframe.row:
    
#     broadC = sc.broadcast(center_rows)
#     xrow = clean_X.collect()[1]
#     '''
#     x = np.array(xrow)
#     centers = broadC.value
#     the_ds = np.zeros(len(centers))
    
#     for c in range(len(centers)):
#         c_array = np.array(centers[c])
#         dist2 = np.linalg.norm(x - c_array)**2
#         the_ds[c] = dist2
        
#     return np.min(the_ds)

# def find_closest_center(xrow,broadC):
#     '''Distance between a dataframe.row
#     and the broadcasted list of centers
#     in the form of dataframe.row:
    
#     broadC = sc.broadcast(center_rows)
#     xrow = clean_X.collect()[1]
#     '''
#     x = np.array(xrow)
#     centers = broadC.value
#     the_ds = np.zeros(len(centers))
    
#     for c in range(len(centers)):
#         c_array = np.array(centers[c])
#         dist2 = np.linalg.norm(x - c_array)**2
#         the_ds[c] = dist2
        
#     return np.argmin(the_ds)

# # Define the function for the logarithm of the cost phi
# def phi(df,centers ):
#     '''logarithm of the cost phi'''
#     return np.log( df.rdd.map(lambda row: distance(row,centers)).reduce(lambda x,y: x + y) )

# # Is this reasonable?? --> maybe we should cast it to int ?
# def evaluate_l(log_phi, k, G):
#     return G * k/log_phi # G = over-oversampling factor

# # Function to select a row based on its probability
# def select_row(x):
#     if x > np.random.uniform(low = 0, high = 1):
#         return True
#     else:
#         return False

In [ ]:
# OPTIMIZED FUNCTIONS

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

# Define the function for the distance between the cluster centers and the pandas dataframe
# @udf('float')
def distance(xrow, centers, delete_last=False):
    '''Distance between a dataframe.row
    and the broadcasted value list of centers
    in the form of dataframe.row.value:
    
    broadC = sc.broadcast(center_rows).value
    xrow = clean_X.collect()[1]
    '''
    if delete_last:
        x = np.array(xrow)[:n_cols]
    else:
        x = np.array(xrow)
    the_ds = np.zeros(len(centers))
    
    for c in range(len(centers)):
        c_array = np.array(centers[c])[:n_cols]
        dist2 = np.linalg.norm(x - c_array)**2
        the_ds[c] = dist2
        
    return np.min(the_ds)

def find_closest_center(xrow,broadC):
    '''Distance between a dataframe.row
    and the broadcasted list of centers
    in the form of dataframe.row:
    
    broadC = sc.broadcast(center_rows)
    xrow = clean_X.collect()[1]
    '''
    x = np.array(xrow)
    centers = broadC.value
    the_ds = np.zeros(len(centers))
    
    for c in range(len(centers)):
        c_array = np.array(centers[c])
        dist2 = np.linalg.norm(x - c_array)**2
        the_ds[c] = dist2
        
    return np.argmin(the_ds)

# # Define the function for the logarithm of the cost phi
# def phi(df,centers ):
#     '''logarithm of the cost phi
#     Remember:
#     centers = broadc.value'''
#     return np.log( df.rdd.map(lambda row: distance(row,centers)).reduce(lambda x,y: x + y) )

# Is this reasonable?? --> maybe we should cast it to int ?
def evaluate_l(log_phi, k, G):
    return G * k/log_phi # G = over-oversampling factor

# Function to select a row based on its probability
def select_row(x):
    if x > np.random.uniform(low = 0, high = 1):
        return True
    else:
        return False
    
def if_greater(xrow, centers):
    return min(distance(xrow, centers, delete_last=True), xrow.__getitem__('minimum_cost'))

### This is NOT $\Phi$. This is $log(\Phi)$. Pay attention budeo.

# Step 1: Choose a random sample from the dataset
This is the required step to begin the algorithm (doesn't need to be parallelized, since it is a select task)

In [ ]:
# Choose the first sample randomly: select the random row
random_n = [np.random.randint(0, n_rows)]
random_sample = getrows(clean_X, random_n).collect()

In [ ]:
# Broadcasting the row over the workers
bCent = sc.broadcast(random_sample)

# To show the content of the broadcast:
# b.value

# Step 2: Initial cost
Then we evaluate the cost function (sum of the squares after the first selection)

In [ ]:
colnames = list(clean_X.dtypes[i][0] for i in range(len(clean_X.dtypes)))

In [ ]:
clean_X = (clean_X.select('*')
           .withColumn('minimum_cost', sum((col(colname)-random_sample[0][colname])**2 for colname in colnames)))

In [ ]:
# Initial cost
# initial_cost = phi(clean_X, bCent)
initial_cost = np.log(clean_X.agg({"minimum_cost": "sum"}).collect()[0][0])
# initial_cost = clean_X.select(['minimum_cost']).rdd.map(lambda x : x).reduce(sum)
initial_cost

## Step 2.5: evaluate number of iterations

After having evaluated $\log \phi$, we find a number of iterations which is of the same order of magnitude.

In [ ]:
int(initial_cost)

In [ ]:
n_iter = int(initial_cost)

# Step 3: For loop
Implement the for loop in order to evaluate probabilities and choose new centers



In [ ]:
phi_iter = initial_cost
l = evaluate_l(phi_iter, k, G)

In [ ]:
l

In [ ]:
clean_X.count()

In [ ]:
import time

In [ ]:
# %%time

# i = 0
# start_time = time.time()

# while i < n_iter or len(bCent.value) < k:
# # while i < 1:
#     '''
#     Nel ciclo for:
#         - Evaluate for each row l * d()^2 / phi
#         - Sample with that probability
#         - Broadcast centers to nodes
#         - Evaluate new cost
        
#     Ricordiamoci che distance è già al quadrato e che phi è il logaritmo
#     '''

#     # Evaluate the probability and select the new rows
#     #probabilities = clean_X.rdd.map(lambda row: np.exp(np.log(distance(row,bCent))-phi_iter)*l)
#     new_rows = clean_X.rdd\
#     .filter(lambda row: select_row(np.exp(np.log(distance(row,bCent))-phi_iter)*l)).collect()
#     print("--- %s seconds ---" % (time.time() - start_time))
#     start_time = time.time()

#     # Update the broadcast
#     bCent = sc.broadcast(bCent.value + new_rows)
#     print("--- %s seconds ---" % (time.time() - start_time))
#     start_time = time.time()

#     # Evaluate new cost
#     phi_iter =  phi(clean_X, bCent)
#     print("--- %s seconds ---" % (time.time() - start_time))
#     start_time = time.time()
    
#     i += 1
#     print("\n at iteration", i ,", #values: ",len(bCent.value), " \n")
# 

for each row evaluate distance from new points --> if it's less than mindistance substitute

In [ ]:
clean_X.printSchema()

In [ ]:
# 4 nuovi centri --> fai una colonna per ogni centro --> modifica --> droppa

In [ ]:
# best to ship this function to all executors in case of huge datasets
def chooseNewDistance(groupedRows, index):

    res = []
    
    for row in groupedRows:
    #calcola nuova distanza da centri --> vedi se è minore dell'ultima entrata --> sostituisci se serve
        new_distance = distance(row, bCent.value[index: ], True)
        best_distance = new_distance
        if row.minimum_cost < best_distance:
            best_distance = row.minimum_cost


    res.append([item for item in row][0:n_cols] + [best_distance])

    return res

In [ ]:
%%time

# OPTIMIZED LOOP

from pyspark.sql.functions import array

i = 0
last_centers = 1
start_time = time.time()

index = 1

while i < n_iter:# or len(bCent.value) < k:

    '''
    Nel ciclo for:
        - Evaluate for each row l * d()^2 / phi
        - Sample with that probability
        - Broadcast centers to nodes
        - Evaluate new cost
        
    Ricordiamoci che distance è già al quadrato e che phi è il logaritmo
    '''
    
    # Evaluate the probability and select the new rows
    new_rows = clean_X.rdd\
               .filter(lambda row: select_row(np.exp(np.log(row.__getitem__('minimum_cost'))-phi_iter)*l)).collect()
    print("--- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()

    # Update the broadcast
    bCent = sc.broadcast(bCent.value + new_rows)
    print("--- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    

    # Update the minimum distance
    if len(new_rows) > 0:
        print("claisjnflkjasdnf")
        data_vals = clean_X.rdd. \
    groupBy(lambda gk: 1). \
    flatMapValues(lambda r: chooseNewDistance(r, index)). \
    values()
        
        data_schema = clean_X.schema
        
        clean_X = spark.createDataFrame(data_vals, data_schema)

        
        
        
        #print('OOOOO', len(new_rows))
        #clean_X = clean_X.withColumn('dummy', 
         #         array(sum((col(colname)-bCent.value[center][colname])**2 for colname in colnames) for center in range(len(new_rows))))

        #print('OOOOO', len(new_rows))
        #clean_X = clean_X.withColumn('dummy', least(*[col("dummy")[i] for i in range(len(new_rows))]))
        
#       # clean_X = clean_X.withColumn('dummy',
#        #           least(*[(sum((col(colname)-bCent.value[center][colname])**2 for colname in colnames) for center in range(len(new_rows)))[i]
#         #          for i in range(len(new_rows))]))

#          #         least(*(sum((col(colname)-bCent.value[center][colname])**2 for colname in colnames) for center in range(len(new_rows)))))

#         #df_with_min = df.withColumn("min_value", least(*[col("values")[i] for i in range(len(data[0][1]))]))

        #print('OOOOO', len(new_rows))
        #clean_X = clean_X.withColumn('minimum_cost', least(col('minimum_cost'), col('dummy')))

        #print('OOOOO', len(new_rows))
        #clean_X = clean_X.drop('dummy')

        #last_centers = len(bCent.value)

#         names = ['minimum_cost']
        
#         for center in range(len(new_rows)):
#             name = 'dummy_'+str(center)
#             clean_X = clean_X.withColumn( name, sum((col(colname)-bCent.value[center][colname])**2 for colname in colnames) )
#             names.append(name)
            
#         column_list = (col(n) for n in names)
#         clean_X = clean_X.withColumn('minimum_cost', least(*column_list))

#         for center in range(len(new_rows)):
#             name = 'dummy_'+str(center)
#             clean_X = clean_X.drop( name )

#         last_centers = len(bCent.value)
        
        # Evaluate new cost
        phi_iter = np.log(clean_X.agg({"minimum_cost": "sum"}).collect()[0][0])

        
    print("--- %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    index = len(bCent.values)

    i += 1
    print("\n at iteration", i ,", #values: ",len(bCent.value), " \n")

In [ ]:
bCent.value[5:]

In [ ]:
phi_iter

In [ ]:
len(bCent.value)

# Step 4: Select a subset of the possible centroids using k-means ++

Using kmeans ++:

Algorithm 1 k-means++(k) initialization.
1: C ← sample a point uniformly at random from X
2: while |C| < k do
2
3:
Sample x ∈ X with probability dφX(x,C)
(C)
4:
C ← C ∪ {

In [ ]:
bCent_list = bCent.value

In [ ]:
dfCent = spark.createDataFrame(bCent_list)

In [ ]:
over_sampled_centers = dfCent.count()

In [ ]:
bCent.unpersist()

In [ ]:
dfCent.rdd.getNumPartitions()

In [ ]:
bCent = sc.broadcast(bCent_list)

In [ ]:
len(bCent.value)

## Reduce size of dfCent to have only `k` centroids 

#### Step 1: calc weights $w_x$
set wx to be the number of points in X closer to x than any other point in C

In [ ]:
from pyspark.sql.functions import count

In [ ]:
wx = clean_X.rdd.map(lambda row: (find_closest_center(row,bCent),1)).reduceByKey(lambda x,y: x+y).takeOrdered(over_sampled_centers) #o lambda x,y: x+y

In [ ]:
wx = (np.array(wx)[:,1]).astype(float)

In [ ]:
wx /= np.sum(wx)
wx

#### Step 2: Use weighted `k-means++`
    2.1 Draw 1 random center
    2.2 update cost function
    2.3 repeat 1-2 until happy

In [ ]:
first_center = np.random.choice(a=range(wx.shape[0]), size=1, p=wx) 
#first point sampled uniformly wrt distance
first_center

In [ ]:
bCent_ultimate = sc.broadcast(getrows(dfCent,first_center).collect())
bCent_ultimate.value

In [ ]:
phi0 = phi(dfCent, bCent_ultimate)
phi0

In [ ]:
type(bCent_ultimate.value)

In [ ]:
%%time

i = 0
while i < k: #n_iter or len(bCent.value) < k:
    '''
    Nel ciclo for:
        - Evaluate for each row l * d()^2 / phi
        - Sample with that probability
        - Broadcast centers to nodes
        - Evaluate new cost
        
    Ricordiamoci che distance è già al quadrato e che phi è il logaritmo
    '''
    
    # Evaluate the probability and select the new rows
    #probabilities = clean_X.rdd.map(lambda row: np.exp(np.log(distance(row,bCent))-phi_iter)*l)
    rows_prob = np.array(dfCent.rdd\
    .map(lambda row: (np.exp(np.log(distance(row,bCent_ultimate))-phi0))).collect())
    
    #sample new random center
    if i == 0:
        print('rows_prob@wx \n',rows_prob@wx, '\n')
    
    another_index = np.random.choice(a=range(wx.shape[0]), size=1, p = rows_prob*wx/(rows_prob@wx) ) 
    
    print(i ,'step: another_index = ', another_index,'\n')
    
    another_center = getrows(dfCent, another_index).collect()
    
    # Update the broadcast
    bCent_ultimate = sc.broadcast(bCent_ultimate.value + another_center)
    
    # Evaluate new cost
    phi0 = phi(dfCent, bCent_ultimate)
    
    i += 1
    print("\n i: ",i,"; bC.val =",bCent_ultimate.value, " \n")

In [ ]:
sum(rows_prob*wx)

In [ ]:
'''
- Prendere la lista dei centroidi e farla diventare un distributed dataset
- Fare una NUOVA lista di centroidi 
- for i in range(k):
    pesca un singolo dato con probabilità d(x)/phi --> come fare?
    aggiungilo alla lista

'''

# TO DO:
    - Capire come scegliere l
    - Fare la normalizzazione dei dati (se la facciamo)
    - Capire se abbiamo ottimizzato nel modo giusto
    - Implementare tutto k means (optional)
    - Creare un file gemello con un dataframe visualizzabile e vedere se ha senso
    - Iniziare a fare i benchmark su cloud veneto
    - Grafici etc...

# Stop worker and master
Stop the running Spark context (sc) and Spark session (spark)

In [ ]:
# sc.stop()
# spark.stop()

Finally, use `docker compose down` to stop and clear all running containers.